# Régression Linéaire

Dans un modèle linéaire, nous supposons que la valeur à prédire $y$ est une combinaison linéaire des variables d'entrée $ X \in R^p $ plus un bruit $ \epsilon $. Mathématiquement, cela peut être exprimé comme :

$ y = \beta_0 + \beta_1 X + \epsilon $

où :
- $ y $ est la variable dépendante (la valeur à prédire),
- $ X $ est le vecteur de valeurs représentant les variables d'entrée,
- $ \beta_0 $ est l'ordonnée à l'origine,
- $ \beta_1 $ est le vecteur des coefficients pour les variables d'entrée,
- $ \epsilon $ est le terme d'erreur (bruit).


Nous pouvons supposer que $ \beta_0 = 0 $ et $ E(\epsilon) = 0$, car nous pouvons simplement étendre $ X $ à $R^{p+1}$, en ajoutant une colonne de uns qui pourrait correspondre à la somme de l'ordonnée à l'origine et de l'espérance du terme d'erreur.

## Estimateur des Moindres Carrés

In [ ]:
# importations
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# définir la graine aléatoire
np.random.seed(0)

# nombre d'échantillons
n = 100

# nombre de variables d'entrée
p = 1

# données d'entrée supposées suivre une distribution normale
# la première colonne est composée de uns (ordonnée à l'origine + espérance de l'erreur)
# nous définissons les données d'entrée avec une moyenne de 0 et un écart-type de 4
sigma = 4
X = np.concatenate((np.ones(n)[:, None], (sigma * np.random.randn(n, p))), axis=1)

# générer le terme d'erreur
# nous définissons le terme d'erreur avec une moyenne de 0 et un écart-type de 3
epsilon = 3*np.random.randn(n)

# définir beta
beta = np.array([3.2, 2.1])

# générer la variable de réponse
y = X.dot(beta) + epsilon

In [ ]:
# étiquettes des axes
plt.xlabel('X')
plt.ylabel('y')

# tracer les données
plt.scatter(X[:, 1], y)
plt.show()

Nous pouvons déduire les estimations des moindres carrés des coefficients $ \beta_0 $ et $ \beta_1 $ en utilisant les formules suivantes :
$ \beta^*= (X^TX)^{-1}X^Ty$

In [ ]:
# déduire la pente et l'ordonnée à l'origine
beta_hat = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)

# afficher la pente et l'ordonnée à l'origine estimées
print('Pente estimée: ', beta_hat[1])
print('Ordonnée à l\'origine estimée: ', beta_hat[0])

# afficher la pente et l'ordonnée à l'origine réelles
print('Pente réelle: ', beta[1])
print('Ordonnée à l\'origine réelle: ', beta[0])

# tracer les données et la droite ajustée
plt.xlabel('X')
plt.ylabel('y')
plt.scatter(X[:, 1], y)
plt.plot(X[:, 1], X.dot(beta_hat), color='red')
plt.show()



## Test de Fisher

Nous voulons tester l'hypothèse nulle selon laquelle le coefficient $ \beta_1 $ est égal à zéro. Nous pouvons utiliser le test de Fisher pour cela. La statistique de test est donnée par :

$ F = \frac{R^2\times(n-p-1)}{(1-R^2)p} $ 

avec $ R^2 = \frac{\Sigma_i(\beta^*.X_i - \bar{y})^2}{\Sigma_i(\beta^*.X_i - y_i)^2} $



In [ ]:
from scipy.stats import f


# calculer les résidus
R_2 = ((y.mean() - X.dot(beta_hat))**2).sum() / ((y - y.mean())**2).sum()

# calculer la statistique F
F = R_2 * (n - p - 1) / ((1 - R_2) * p)

# afficher les résultats
print(f"Le seuil de rejet est : {f.ppf(0.95, p, n - p - 1)}")
print(f"La statistique F est : {F}")
print(f"La p-valeur est : {1 - f.cdf(F, p, n - p - 1)}")


Nous construisons un cadre où l'hypothèse nulle est vraie. Nous générons un ensemble de données avec une relation linéaire entre la variable d'entrée et la variable de sortie, et nous testons si le test de Fisher peut rejeter l'hypothèse nulle.

In [ ]:
# y ne dépend pas de la deuxième colonne de X
y_test_2 = epsilon

# déduire la pente et l'ordonnée à l'origine
beta_hat_2 = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y_test_2)

# calculer les résidus
R_2_2 = ((y_test_2.mean() - X.dot(beta_hat_2))**2).sum() / ((y_test_2 - y_test_2.mean())**2).sum()

# calculer la statistique F
F_2 = R_2_2 * (n - p - 1) / ((1 - R_2_2) * p)

# afficher les résultats
print(f"Le seuil de rejet est : {f.ppf(0.95, p, n - p - 1)}")
print(f"La statistique F est : {F_2}")
print(f"La p-valeur est : {1 - f.cdf(F_2, p, n - p - 1)}")

## Test de Student

Nous pouvons également tester l'hypothèse nulle selon laquelle le coefficient $ \beta_1 $ est égal à zéro en utilisant le test de Student. La statistique de test est donnée par :

$ t = \frac{\beta_1}{\sigma_1} $


# Rendu de TP

Générez trois cas :
- Un cas où l'hypothèse nulle est vraie pour toutes les composantes, $p=2$ et $n=100$. 
- Un cas où l'hypothèse nulle est vraie pour une composante, $p=2$ et $n=100$.
- Un cas où l'hypothèse nulle est fausse pour toutes les composantes, $p=2$ et $n=100$.

Fixez la graine à $0$ à des fins de reproductibilité.


# 